# Handle Missing Data

In [2]:
import findspark
findspark.init('/home/aforestier10/Downloads/spark-3.5.3-bin-hadoop3')

In [3]:
import pyspark
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Agg').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

### Read in Data

In [6]:
df = spark.read.csv('ContainsNull.csv', header=True, inferSchema=True)

In [7]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Drop Data

In [9]:
# Drops any row w/ missing data
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [11]:
# Set threshold of non null values to not drop
# in this example, row must have at least 2 non-null values to not be dropped
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
# Can specify how - i.e. 'all' means all must be null
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| NULL|
|emp2| NULL| NULL|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
# Subset - only check these coluumns
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



### Fill in missing values

In [16]:
# Spark is type smart. Will only fill in matching values
df.na.fill('FILL VALUES').show()

+----+-----------+-----+
|  Id|       Name|Sales|
+----+-----------+-----+
|emp1|       John| NULL|
|emp2|FILL VALUES| NULL|
|emp3|FILL VALUES|345.0|
|emp4|      Cindy|456.0|
+----+-----------+-----+



In [17]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| NULL|  0.0|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [18]:
# Specify col
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| NULL|
|emp2|No Name| NULL|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [26]:
# Use average
avg_sales = df.agg({'Sales': 'avg'}).collect()
avg_sales

[Row(avg(Sales)=400.5)]

In [27]:
avg_sales = avg_sales[0][0]
avg_sales

400.5

In [29]:
df.na.fill(avg_sales, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| NULL|400.5|
|emp3| NULL|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

